### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [1]:
import pandas as pd
import numpy as np
import pickle

import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_yards, get_contribution, get_touchdowns, LogShift, stats_for_trailing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore',
                     category = SettingWithCopyWarning)

### Import Data

Let's import the dataframe that we will be using for modelling

In [3]:
data = pd.read_csv('Data/weekly_data.csv')

data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0.0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1.0,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2.0,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4.0,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5.0,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32046 entries, 0 to 32045
Data columns (total 65 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    32046 non-null  int64  
 1   Unnamed: 0.1                  26551 non-null  float64
 2   GameKey                       32046 non-null  int64  
 3   PlayerID                      32046 non-null  int64  
 4   Season                        32046 non-null  int64  
 5   GameDate                      32046 non-null  object 
 6   Week                          32046 non-null  int64  
 7   Team                          32046 non-null  object 
 8   Opponent                      32046 non-null  object 
 9   HomeOrAway                    32046 non-null  object 
 10  Number                        32046 non-null  int64  
 11  Name                          32046 non-null  object 
 12  Position                      32046 non-null  object 
 13  S

In [4]:
data['Week'] = data['Week'].astype(str)

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [5]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df.loc[:,column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    df_out.sort_index(inplace = True)
    return df_out
    

In [6]:
stats_for_trailing = ['FantasyPointsPPR']

In [7]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data[data['Name'] == 'Tom Brady'][['FantasyPointsPPR', 'TAFantasyPointsPPR']].head(35)

,FantasyPointsPPR,TAFantasyPointsPPR
14,21.28,18.332
506,16.36,18.332
1077,7.52,18.332
1454,18.96,18.332
1860,27.54,18.332
2325,21.80,18.332
2758,21.68,18.436
3212,13.76,19.500
3665,15.66,20.748
4128,11.76,20.088


### Train Test Split

In [8]:
# Separate data from the target
# y = data['FantasyPointsPPR']

# Apply a log transform to the target to see how it impacts prediction
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [5]:
data

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0.0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1.0,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2.0,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4.0,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5.0,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32041,103457,NaN,202111108,20283,2021,2021-11-21T13:00:00,11,DET,CLE,AWAY,...,0.0,0.0,NaN,202111DETCLE,CLE,NaN,NaN,DET,NaN,NaN
32042,103458,NaN,202111104,18811,2021,2021-11-21T13:00:00,11,BUF,IND,HOME,...,0.0,0.0,NaN,202111INDBUF,BUF,NaN,NaN,IND,NaN,NaN
32043,103459,NaN,202111102,19763,2021,2021-11-18T20:20:00,11,ATL,NE,HOME,...,0.0,0.0,NaN,202111NEATL,ATL,0.0,165.0,NE,25.0,308.0
32044,103460,NaN,202111102,22498,2021,2021-11-18T20:20:00,11,ATL,NE,HOME,...,0.0,0.0,NaN,202111NEATL,ATL,0.0,165.0,NE,25.0,308.0


In [9]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.2,
                                                    shuffle = False,
                                                   random_state = 13)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [10]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats.append('TAFantasyPointsPPR')

In [11]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per week per player
yard_transformer = FunctionTransformer(get_yards) # Get total yardage per week per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance
contribution_transformer = FunctionTransformer(get_contribution) # Calculate the offensive contribution of a given player relative to the team's offense


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                        ('yards', yard_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer),
                       ('contribution', contribution_transformer)])

<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [12]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                       'Name',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']

numerical_columns = trailing_stats


In [13]:
# Create a custom function to generate a log-transformed version of continuous data with a constant 5 added prior to the transform

LogShiftTransformer = FunctionTransformer(LogShift)

In [14]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder(handle_unknown = 'ignore'))])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'mean')),
                               ('log', LogShiftTransformer)])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [15]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', Ridge())])

In [16]:
# Set param grid values, parameters for grid search 

param_grid = {'prep__num__log': [RobustScaler(), StandardScaler(), LogShiftTransformer],
             'model__alpha': [1]}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                           scoring = 'neg_mean_squared_error',
                          cv = 5,
                          verbose = 3)

In [ ]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END model__alpha=1, prep__num__log=RobustScaler();, score=-31.046 total time=  39.2s
[CV 2/5] END model__alpha=1, prep__num__log=RobustScaler();, score=-31.119 total time=  47.1s
[CV 3/5] END model__alpha=1, prep__num__log=RobustScaler();, score=-28.892 total time=  50.2s
[CV 4/5] END model__alpha=1, prep__num__log=RobustScaler();, score=-32.005 total time=  59.9s
[CV 5/5] END model__alpha=1, prep__num__log=RobustScaler();, score=-27.162 total time=  42.8s
[CV 1/5] END model__alpha=1, prep__num__log=StandardScaler();, score=-33.833 total time=  41.4s
[CV 2/5] END model__alpha=1, prep__num__log=StandardScaler();, score=-31.256 total time=  39.0s
[CV 3/5] END model__alpha=1, prep__num__log=StandardScaler();, score=-30.578 total time=  40.8s
[CV 4/5] END model__alpha=1, prep__num__log=StandardScaler();, score=-30.993 total time=  41.9s
[CV 5/5] END model__alpha=1, prep__num__log=StandardScaler();, score=-29.325 total tim

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
preds = grid_search.predict(X_test)

In [ ]:
mean_squared_error(y_test, preds)

In [ ]:
mean_absolute_error(y_test, preds)


In [ ]:
r2_score(y_test, preds)

Let's dig into the predictions a little bit

In [ ]:
df_preds = pd.DataFrame(y_test)
df_preds['Predicted'] = preds
df_preds

### Save model


In [ ]:
import pickle

pickle.dump(grid_search, open('Pickles/log_transform_linear.pickle', 'wb'))

# Time to dig into the results

In [ ]:
df_review = X_test
df_review['Predicted'] = df_preds['Predicted']
df_review['Target'] = df_preds['FantasyPointsPPR']
df_review['Error'] = df_review['Target'] - df_review['Predicted']
df_review.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Let's visualize how the prediction error varies by player position.

plt.figure(figsize = (12, 9))
sns.violinplot(x = df_review['Position'],
               y = df_review['Error'])

In [ ]:
# Let's apply our engineered features to df_review and keep on digging
df_review = trailing_stats_mean(df_review)
df_review['PlayerTier'] = df_review.apply(lambda x: tier_maker(x['Position'], x['TAFantasyPointsPPR']), axis = 1)


In [ ]:
df_review['AbsoluteError'] = abs(df_review['Error'])
mean_tier_error = df_review[['PlayerTier', 'Error', 'AbsoluteError']].groupby('PlayerTier').mean()
sum_tier_error = df_review[['PlayerTier', 'Error','AbsoluteError']].groupby('PlayerTier').sum()
count_tier_error = df_review[['PlayerTier', 'Error']].groupby('PlayerTier').count()
mean_tier_error['Total Absolute Error'] = sum_tier_error['AbsoluteError']
mean_tier_error['Total Error'] = sum_tier_error['Error']
mean_tier_error['Error Count'] = count_tier_error['Error']
mean_tier_error.rename({'Error': 'Mean Error'}, 
                       axis = 1)